In [ ]:
import pandas as pd
import numpy as np
import random

# Import the Excel file
file_path = 'D:\\GPData.xlsx'
sheet_name = 'Database'

# Read the data
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Define problem parameters
n = len(df)  # Number of tour packages
m = 4  # Number of seasons (assuming Winter, Spring, Summer, Autumn)

# Extract data from the dataframe
price = df['Price'].values
cost_package_per_tourist = df['Cost of Package Per Tourist'].values
cost_per_unit_resource = df['Cost per unit of resources'].values
actual_demand = df['Actual Demand'].values
min_demand = df['Minimum Demand'].values
max_demand = df['Maximum Demand'].values
resources_per_package = df['#Resources of Package'].values
available_resources = df['#Available Resources'].values
season_offered = df['Season'].values

# Constants
br = 100000  # Total budget of the company
mij = 5  # Maximum number of packages i to offer per season

seasons = ['Winter', 'Spring', 'Summer', 'Autumn']

def initialize_population(pop_size, n, m):
    return np.random.randint(2, size=(pop_size, n, m))

def fitness(individual):
    total_revenue = 0
    total_cost = 0
    total_profit = 0
    total_resource_cost = 0
    total_resources_used = np.zeros(m)
    total_packages_offered = np.zeros(m)
    
    for i in range(n):
        for j in range(m):
            if individual[i, j] == 1 and seasons[j] in season_offered[i]:
                revenue = price[i] * actual_demand[i]
                cost_packages = cost_package_per_tourist[i] * actual_demand[i]
                cost_resources = cost_per_unit_resource[i] * resources_per_package[i]
                
                total_cost += cost_packages + cost_resources
                total_resource_cost += cost_resources
                total_revenue += revenue
                total_resources_used[j] += resources_per_package[i]
                total_packages_offered[j] += 1
            else:
                continue

    total_profit = total_revenue - total_cost
    
    # Apply budget constraint
    if total_resource_cost > br:
        total_profit -= np.inf  # Penalize solutions that exceed budget constraints

    # Apply resource constraints
    for k in range(m):
        if total_resources_used[k] > available_resources[k]:
            total_profit -= np.inf  # Penalize solutions that exceed resource constraints

    # Apply seasonal constraints
    for j in range(m):
        if total_packages_offered[j] > mij:
            total_profit -= np.inf  # Penalize solutions that exceed maximum package offerings per season
    
    return total_profit

def selection(population, fitnesses):
    selected_indices = np.argsort(fitnesses)[-2:]  # Select top 2 individuals
    return population[selected_indices]

def crossover(parent1, parent2):
    cross_point = random.randint(0, n * m - 1)
    child1 = np.concatenate((parent1.flatten()[:cross_point], parent2.flatten()[cross_point:])).reshape(n, m)
    child2 = np.concatenate((parent2.flatten()[:cross_point], parent1.flatten()[cross_point:])).reshape(n, m)
    return child1, child2

def mutation(individual, mutation_rate=0.01):
    for i in range(n):
        for j in range(m):
            if random.random() < mutation_rate:
                individual[i, j] = 1 - individual[i, j]
    return individual

def genetic_algorithm(pop_size, generations, mutation_rate):
    population = initialize_population(pop_size, n, m)
    best_solution = None
    best_fitness = -np.inf
    
    for generation in range(generations):
        fitnesses = np.array([fitness(individual) for individual in population])
        total_profit = np.max(fitnesses)
        print(f"Generation {generation + 1}: Total Profit = {total_profit:.2f}")
        
        if total_profit > best_fitness:
            best_fitness = total_profit
            best_solution = population[np.argmax(fitnesses)].copy()
        
        new_population = []
        
        # Elitism: Keep the best individual from the current generation
        elite = population[np.argmax(fitnesses)]
        new_population.append(elite)
        
        for _ in range((pop_size - 1) // 2):
            parents = selection(population, fitnesses)
            child1, child2 = crossover(parents[0], parents[1])
            child1 = mutation(child1, mutation_rate)
            child2 = mutation(child2, mutation_rate)
            new_population.append(child1)
            new_population.append(child2)
        
        # Ensure the population size remains constant
        population = np.array(new_population[:pop_size])
        
        # Ensure the best solution seen so far is carried to the next generation
        if best_solution is not None:
            population[-1] = best_solution
    
    return best_solution, best_fitness

def print_solution(solution):
    total_profit_calculated = 0
    
    for i in range(n):
        for j in range(m):
            if seasons[j] in season_offered[i]:
                if solution[i, j] == 1:
                    if actual_demand[i] < min_demand[i]:
                        print(f"Package {df['Tour Name'][i]} Not Offered in Season {seasons[j]}")
                        print(f"As Actual Demand is Less Than Minimum Demand and it violates Minimum Demand Constraint")
                        print(f"Actual Demand: {actual_demand[i]}")
                        print(f"Min Demand: {min_demand[i]}")
                        print(f"Max Demand: {max_demand[i]}\n")
                    elif actual_demand[i] > max_demand[i]:
                        print(f"Package {df['Tour Name'][i]} Not Offered in Season {seasons[j]}")
                        print(f"As Actual Demand is Greater Than Maximum Demand and it violates Maximum Demand Constraint")
                        print(f"Actual Demand: {actual_demand[i]}")
                        print(f"Min Demand: {min_demand[i]}")
                        print(f"Max Demand: {max_demand[i]}\n")
                    else:
                        revenue = price[i] * actual_demand[i]
                        cost_packages = cost_package_per_tourist[i] * actual_demand[i]
                        cost_resources = cost_per_unit_resource[i] * resources_per_package[i]
                        total_cost = cost_packages + cost_resources
                        total_profit = revenue - total_cost
                        total_profit_calculated += total_profit
                        print(f"Package {df['Tour Name'][i]} Offered in Season {seasons[j]}")
                        print(f"Price: ${price[i]}")
                        print(f"Actual Demand: {actual_demand[i]}")
                        print(f"Min Demand: {min_demand[i]}")
                        print(f"Max Demand: {max_demand[i]}")
                        print(f"Total Revenue: ${revenue:.2f}")
                        print(f"Total Cost: ${total_cost:.2f}")
                        print(f"Total Profit: ${total_profit:.2f}\n")
                else:
                    reason = []
                    total_resource_cost = cost_per_unit_resource[i] * resources_per_package[i]
                    total_resources_used = np.zeros(m)
                    total_packages_offered = np.zeros(m)
                    
                    for k in range(m):
                        if solution[i, k] == 1:
                            total_resources_used[k] += resources_per_package[i]
                            total_packages_offered[k] += 1
                    
                    if total_resources_used[j] > available_resources[j]:
                        reason.append(f"Resource constraint violated")
                    if total_resource_cost > br:
                        reason.append(f"Budget constraint violated")
                    if total_packages_offered[j] > mij:
                        reason.append(f"Seasonal constraint violated")
                    if actual_demand[i] < min_demand[i]:
                        reason.append(f"Actual Demand is less than Minimum Demand")
                    if actual_demand[i] > max_demand[i]:
                        reason.append(f"Actual Demand is greater than Maximum Demand")
                    reason_text = ", ".join(reason) if reason else "Not selected by the GA"
                    print(f"Package {df['Tour Name'][i]} Not Offered in Season {seasons[j]}")
                    print(f"Reason: {reason_text}")
                    print(f"Actual Demand: {actual_demand[i]}")
                    print(f"Min Demand: {min_demand[i]}")
                    print(f"Max Demand: {max_demand[i]}\n")
    
    print(f"Calculated Total Profit for Best Solution: ${total_profit_calculated:.2f}")

# Example: Running GA with population size 50, 100 generations, and mutation rate 1%
best_solution, best_fitness = genetic_algorithm(50, 100, 0.01)
print_solution(best_solution)
print(f"Best Total Profit: {best_fitness:.2f}")
